In [3]:
import timeit

from jupyter_core.version import parts

dict0 = {'a': ['1', '2'], 'b': ['+', '*'], 'c': ['!', '@']}
 # Unvectorized cart product
for a in dict0['a']:
    for b in dict0['b']:
        for c in dict0['c']:
            print({'a': a, 'b': b, 'c': c})


{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': '@'}
{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': '@'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': '@'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': '@'}


In [4]:
from itertools import product
# Vectorized Cartegian product
dict0 = {"a": ["1", "2"], "b": ["*", "+"], "c": ["!", "@"]}
jobs = (dict(zip(dict0, i)) for i in product(*dict0.values()))

for i in jobs:
    print(i)


{'a': '1', 'b': '*', 'c': '!'}
{'a': '1', 'b': '*', 'c': '@'}
{'a': '1', 'b': '+', 'c': '!'}
{'a': '1', 'b': '+', 'c': '@'}
{'a': '2', 'b': '*', 'c': '!'}
{'a': '2', 'b': '*', 'c': '@'}
{'a': '2', 'b': '+', 'c': '!'}
{'a': '2', 'b': '+', 'c': '@'}


In [5]:
import numpy as np

def barrierTouch(sub_r, width=0.5):
    t = {}
    p = np.log(np.cumprod(1 + sub_r, axis=0))  # cumulative log price

    for j in range(p.shape[1]):
        for i in range(p.shape[0]):
            if abs(p[i, j]) >= width:
                t[j] = i
                break
    return t



def main0():
    r = np.random.normal(0, 0.01, size=(1000, 10000))
    t = barrierTouch(r)
    print(f"{len(t)} paths touched the barrier")
    return t

# Run it
if __name__ == '__main__':
    import timeit
    print (min(timeit.Timer('main0()', setup = 'from __main__ import main0').repeat(1, 10)))

2161 paths touched the barrier
2179 paths touched the barrier
2210 paths touched the barrier
2326 paths touched the barrier
2255 paths touched the barrier
2246 paths touched the barrier
2261 paths touched the barrier
2233 paths touched the barrier
2257 paths touched the barrier
2280 paths touched the barrier
24.039596922000783


In [7]:
import numpy as np

def linParts(numAtoms, numThreads):
    # partition of atoms with a single loop
    parts = np.linspace(0, numAtoms, min(numThreads, numAtoms)+1, dtype=int)
    return parts


In [ ]:
def nestedParts(numAtoms, numThreads, upperTriang=False):
    # partition of atoms with an inner loop (e.g., quadratic load)
    parts = [0]
    numThreads_ = min(numThreads, numAtoms)

    for num in range(numThreads_):
        part = 1 + 4 * (parts[-1]**2 + parts[-1] + numAtoms * (numAtoms + 1.) / numThreads_)
        part = (-1 + part**0.5) / 2
        parts.append(part)

    parts = np.round(parts).astype(int)

    if upperTriang:
        # Reverse differences to mimic upper triangular load balancing
        parts = np.cumsum(np.diff(parts)[::-1])
        parts = np.append(np.array([0]), parts)

    return parts


In [16]:
import multiprocessing as mp

def processJobs(jobs, numThreads=24):
    """
    Process jobs in parallel using multiprocessing.
    :param jobs: A list of job dictionaries.
    :param numThreads: Number of worker processes.
    :return: List of results.
    """
    with mp.Pool(processes=numThreads) as pool:
        out = pool.map(worker, jobs)
    return out

def processJobs_(jobs):
    """
    Process jobs serially (no parallelism).
    :param jobs: A list of job dictionaries.
    :return: List of results.
    """
    out = [worker(job) for job in jobs]
    return out

def worker(job):
    """
    Worker function to execute a job.
    :param job: A dictionary containing 'func' and its arguments.
    :return: Output of the func(**job).
    """
    func = job.pop('func')
    return func(**job)


In [17]:
import pandas as pd

def mpPandasObj(func, pdObj, numThreads=24, mpBatches=1, linMols=True, **kargs):
    """
    Parallelize a pandas job over multiple processes.

    :param func: Target function to apply; must return a DataFrame or Series.
    :param pdObj: Tuple (argName, atoms), where atoms will be partitioned.
    :param numThreads: Number of worker processes.
    :param mpBatches: Number of jobs per core.
    :param linMols: If True, use linear partitioning; else nested.
    :param kargs: Additional keyword args for func.
    :return: Concatenated result of all parallel jobs.
    """
    # Select partitioning method
    atoms = pdObj[1]
    argName = pdObj[0]
    totalJobs = numThreads * mpBatches
    parts = linParts(len(atoms), totalJobs) if linMols else nestedParts(len(atoms), totalJobs)

    # Create jobs
    jobs = []
    for i in range(1, len(parts)):
        job = {argName: atoms[parts[i-1]:parts[i]], 'func': func}
        job.update(kargs)
        jobs.append(job)

    # Execute
    if numThreads == 1:
        out = processJobs_(jobs)
    else:
        out = processJobs(jobs, numThreads=numThreads)

    # Concatenate output
    if isinstance(out[0], pd.DataFrame):
        result = pd.concat(out)
    elif isinstance(out[0], pd.Series):
        result = pd.concat(out)
    else:
        return out  # raw output (e.g. list of scalars)

    return result.sort_index()


In [ ]:
import multiprocessing as mp

In [8]:
import sys
import time
import datetime as dt


def processJobs(jobs):
    # Run jobs sequantially, for debugging
    out = []
    for job in jobs:
        out_ = expandCall(job)
        out.append(out)
    return out

import multiprocessing as mp

def reportProgress(jobNum, numJobs, time0, task):
    #Report progress as asynch jobs are completed
    msg = [float (jobNum) / numJobs, (time.time() -time0)/60.]
    msg.append(msg[1] *(1/msg[0] - 1))
    timeStamp = str(dt.datetime.fromtimestamp(time.time()))
    msg = timeStamp+' '+ str(round(mag[0]*100, 2)) + '% ' + task + ' done after ' +\
        str(round(msg[1], 2)) + ' minutes. Remaining ' + str(round(msg[2], 2))+' minutes.'
    if jobNum <numJobs: sys.stderr.write(msg+'\r')
    else:sys.stderr.write(mag+'\n')
    return